In [1]:
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os

In [2]:
# data load
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/Attempt3_mr_scrape.pickle', 'rb') as handle:
    scrapevar = pickle.load(handle)

In [3]:
# get_proxies():
def get_proxies(nprox):
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    curr_soup = BeautifulSoup(response.text)
    rand_ipadd = []
    alllist = curr_soup.findAll("tr")
    # alllist[1].next.text
    rvec = random.sample(range(1, 15), nprox)
    for i in rvec:
        rand_ipadd.append((alllist[i].next.text + ":" + alllist[i].next.next.next.text))
    return rand_ipadd

def get_user_agents(nagents):
    useragent_add = 'https://developers.whatismybrowser.com/useragents/explore/software_type_specific/web-browser/'
    page = requests.get(useragent_add)
    curr_soup = BeautifulSoup(page.text)
    rand_useragents = []
    alllist = curr_soup.findAll("td", {"class": "useragent"})
    rvec = random.sample(range(0, 25), nagents)
    for i in rvec:
        a = alllist[i].find("a")
        rand_useragents.append(a.text)
    return rand_useragents

def build_proxy_agent(proxy_list,user_agent_list):
    proxy = random.choice(proxy_list)
    http_proxy  = "http://" + proxy
    https_proxy = "https://" + proxy

    proxyDict = { 
                  "http"  : http_proxy, 
                  "https" : https_proxy
                }
    user_agent = random.choice(user_agent_list)
    #Set the headers 
    headers = {'User-Agent': user_agent}
    return proxyDict, headers

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [25]:
article_url = scrapevar['links']['nick-sciria'][1]
# can run headless
# page = requests_retry_session(retries=10).get(
#     article_url, timeout=10)
reader_header = 'about:reader?url='

# create a new Firefox session
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(reader_header + article_url)
time.sleep(1)
soup_level1=BeautifulSoup(driver.page_source)
textsection = soup_level1.find("div", {"class": "content"})
pgraphs = textsection.findAll("p")
alltext = ''
for i in pgraphs:
    alltext = alltext + i.text + ' '
print(alltext)
driver.close()

 BOSTON, MA - MAY 25: Kyrie Irving #2 of the Cleveland Cavaliers and Marcus Smart #36 of the Boston Celtics greet before the game in Game Five of the Eastern Conference Finals of the 2017 NBA Playoffs on May 25, 2017 at the TD Garden in Boston, Massachusetts. NOTE TO USER: User expressly acknowledges and agrees that, by downloading and or using this photograph, User is consenting to the terms and conditions of the Getty Images License Agreement. Mandatory Copyright Notice: Copyright 2017 NBAE (Photo by Brian Babineau/NBAE via Getty Images) Sometimes, the allure of changing scenery can turn out to be nothing more than a hoax. The newness and unfamiliarity that once seemed so appealing unfolds into the same hidden conflicts and particularly unique challenges of old. However, Kyrie Irving’s new home provides him with a distinctly new world of values and responsibilities, features that could make Irving’s change of scenery well worth it. To demonstrate this difference, let’s compare Kyrie 

In [23]:
print(article_url)

https://bleacherreport.com/articles/2780278-still-in-championship-mode-after-frustrating-finals-lebrons-future-is-clear


In [15]:
driver.close()

In [6]:
# attempt to pull espn article
article_url = scrapevar['links']['kevin-arnovitz'][6]
# page = requests_retry_session(retries=10).get(
#     article_url, timeout=10)
reader_header = 'about:reader?url='
print(reader_header + article_url)
page = requests_retry_session(retries=10).get(
                reader_header + article_url,timeout=10)
# page = requests.get(reader_header + article_url)
# curr_soup = BeautifulSoup(page.text)
# textcontainer = curr_soup.find("article", {"class": "article"})
# textsection = textcontainer.find("div", {"class": "article-body"})
# pgraphs = textsection.findAll("p")
# alltext = ''
# for i in pgraphs:
#     alltext = alltext + i.text + ' '
# print(alltext)

about:reader?url=http://www.espn.com/blog/truehoop/post/_/id/56995/memphis-at-clippers-game-2-five-things-to-watch


InvalidSchema: No connection adapters were found for 'about:reader?url=http://www.espn.com/blog/truehoop/post/_/id/56995/memphis-at-clippers-game-2-five-things-to-watch'